<a href="https://colab.research.google.com/github/valkryhx/lora_bnb_int8/blob/main/trlx_sentiments_0416%E7%89%88_%E5%BC%84%E6%B8%85%E4%BA%86%E4%BD%9C%E7%94%A8%E6%9C%BA%E7%90%86.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Produce Movie Reviews with Positive Sentiment

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/CarperAI/trlx/blob/main/examples/notebooks/trlx_sentiments.ipynb)

#### Optimize gpt2 to review movies positively based on a corpus of IMDB reviews with sentiment scores from DistilBert.

Notebook by [@zswitten](https://github.com/zswitten)

---

In [ ]:
# https://github.com/CarperAI/trlx/blob/main/examples/summarize_daily_cnn/t5_summarize_daily_cnn.py

# 这是flan-t5-用来做summarize的例子  可见是可以用的

# 见最下面

[TODO]剩下要做的就是把peft弄进来

[20230417学习经验]

使用google的T5系列以及FLAN-T5系列模型
在使用trlX框架训练时，训练过程会先tokenize samples=imdb['text']的每一条样本文字
比如samples使用了imdb['text'][:20]那么tokenize操作20次
但是目前感觉trlX实现的有问题
会大量报warning
Using bos_token, but it is not set yet.
Using bos_token, but it is not set yet.
Using bos_token, but it is not set yet.
Using bos_token, but it is not set yet.
看issue 讨论 开发者是修复了手动加入special token的bug 但是为何还是会上报 这就不知道了

也就是说 如果样本是20000条 那么这个
Using bos_token, but it is not set yet.
Using bos_token, but it is not set yet.
Using bos_token, but it is not set yet.
Using bos_token, but it is not set yet.
就会print 20000次 很尬 会造成缓冲区胀满

# 但是实际上这些输出执行完毕后 还是能正常训练的！！！


[20230416学习经验]

原先是用POSITIVE值当作value
这一版本我测试把metric_fn中用NEGATIVE值当作value

这样metric_fn输出的logtis有两个位置，正向文字概率和负向情感文字概率，我们现在是把负向情感文字的概率当score

那么除了把字典里面的key换成NEGATIVE之外
还需要把

trainer = trlx.train(
    samples=imdb["text"],  #输入文本
    rewards=[1.0 - value  for value in imdb["label"]],
    . . .

的rewards值用1做差 这样原先negative label=0  ，positive label=1

现在 negative label=1  ，positive label=0 

正好label又作为reward的奖励指引 所以意味着negative奖励为1 大于positive 奖励为0 




训练超参数调整可以直接改key的value 注意default_config['train']['total_steps']才是决定训练步数的

default_config['train']['batch_size'] = 8

default_config['train']['epochs'] = 10

default_config['train']['total_steps'] = 2000

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#%pip install torch==2.0.0
#%pip install -Uqq circuitsvis

In [ ]:
# %pip uninstall numpy
# %pip install numpy==1.23.5
# %pip install torch --extra-index-url https://download.pytorch.org/whl/cu116

In [ ]:
# %pip install -Uqq git+https://github.com/neelnanda-io/TransformerLens
#%pip install -Uqq circuitsvis

In [ ]:
# %pip uninstall numpy
# %pip install numpy==1.23.5


In [ ]:
# %pip uninstall torch

In [ ]:
# %pip install torch==1.13.1+cu116 --extra-index-url https://download.pytorch.org/whl/cu116

In [ ]:
%mkdir -p /content/drive/MyDrive/trlx_0403

In [ ]:
%cd /content/drive/MyDrive/trlx_0403

/content/drive/MyDrive/trlx_0403


Execute the cells below to install [TRLX](https://github.com/CarperAI/trlx) for a colab environment.

In [ ]:

!pip install datasets loralib sentencepiece torchtyping opendelta transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.1/88.1 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 43.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 56.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 623.2/623.2 kB 59.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Prepari

In [ ]:
!git clone https://github.com/CarperAI/trlx.git
#!git config --global --add safe.directory /content/trlx && cd /content/trlx && pip install -e .

Cloning into 'trlx'...
remote: Enumerating objects: 6526, done.
remote: Counting objects: 100% (521/521), done.
remote: Compressing objects: 100% (281/281), done.
remote: Total 6526 (delta 335), reused 376 (delta 238), pack-reused 6005
Receiving objects: 100% (6526/6526), 46.61 MiB | 13.02 MiB/s, done.
Resolving deltas: 100% (4208/4208), done.


进入到trlx源码目录

使用pip install -e . 

以开发模式安装当前目录下的whatever packages
参考[https://www.reddit.com/r/learnpython/comments/ayx7za/how_does_pip_install_e_work_is_there_a_specific/]
"Use the program pip to install whatever package is in the current directory/folder in development mode"

In [ ]:
%cd /content/drive/MyDrive/trlx_0403/trlx
!pip install -e .

/content/drive/MyDrive/trlx_0403/trlx
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/drive/MyDrive/trlx_0403/trlx
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 MB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.3/215.3 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 764.8/764.8 kB 65.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 53.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 74.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# import numpy 
# numpy.__version__

In [ ]:
# uninstall scikit_learn + jax to avoid numpy issues
!pip uninstall -y scikit_learn jax

Found existing installation: scikit-learn 1.2.2
Uninstalling scikit-learn-1.2.2:
  Successfully uninstalled scikit-learn-1.2.2
Found existing installation: jax 0.4.8
Uninstalling jax-0.4.8:
  Successfully uninstalled jax-0.4.8


In [ ]:
import os

# run within repo
os.chdir('/content/drive/MyDrive/trlx_0403/trlx')
print(os.getcwd())

/content/drive/MyDrive/trlx_0403/trlx


In [ ]:
# import wandb
# wandb.init(mode="disabled")

In [ ]:
import yaml
from datasets import load_dataset
from transformers import pipeline,AutoTokenizer,AutoModelForCausalLM
import pathlib
from typing import Dict, List
import trlx
from trlx.data.default_configs import TRLConfig, default_ilql_config

In [ ]:
from trlx.data.configs import (
    ModelConfig,
    OptimizerConfig,
    SchedulerConfig,
    TokenizerConfig,
    TrainConfig,
    TRLConfig,
)
from trlx.models.modeling_ppo import PPOConfig
from trlx.models.modeling_ilql import ILQLConfig

In [ ]:
# default_config = default_ilql_config().to_dict()
# print(type(default_config))
# print(default_config)
# default_config['method']


In [ ]:
# print(default_config)

In [ ]:

# train=TrainConfig(
#         seq_length=64,
#         epochs=100,
#         total_steps=1000,
#         batch_size=1,
#         checkpoint_interval=100,
#         eval_interval=10,
#         pipeline="PromptPipeline",
#         trainer="AccelerateILQLTrainer",
#         tracker=None, #default is wandb
#     )
# model=ModelConfig(
#         model_path="google/flan-t5-small",
#         model_arch_type="seq2seq",
#         num_layers_unfrozen=2,
#     )
# tokenizer=TokenizerConfig(
#         tokenizer_path="google/flan-t5-small",
#         truncation_side="right",
#     )
# optimizer=OptimizerConfig(
#         name="adamw",
#         kwargs={
#             "lr": 1.0e-5,
#             "betas": [0.9, 0.999],
#             "eps": 1.0e-8,
#             "weight_decay": 1.0e-6,
#         },
#     )
# scheduler=SchedulerConfig(
#         name="cosine_annealing",
#         kwargs={
#             "T_max": 10000,
#             "eta_min": 1.0e-6,
#         },
#     )
   


In [ ]:
# default_config = default_ilql_config()
# default_ilql_config().method

In [ ]:
# default_config = default_ilql_config().to_dict()
# default_config['train'] = train
# default_config['model'] = model
# default_config['tokenizer'] = tokenizer
# default_config['optimizer'] = optimizer
# default_config['scheduler'] = scheduler

# #config = TRLConfig.update(default_config, {})
# print(default_config)


# print(config)

In [ ]:
# default_config =default_config.to_dict()
# default_config['train']['tracker'] = None
# default_config['train']['batch_size'] = 8
# default_config['train']['epochs'] = 10
# default_config['train']['total_steps'] = 2000
# #我们换成t5-small 试试
# #  t5:11B gpt2:1.5B t5-small:6000万
# #注意t5 
# default_config['model']['model_path'] = 'google/flan-t5-small'  #model_arch_type =se12seq
# default_config['model']['model_arch_type'] = 'seq2seq'
# default_config['model']['num_layers_unfrozen'] = 2
# default_config["tokenizer"]["padding_side"] = "right"
# default_config["tokenizer"]["tokenizer_path"] ='google/flan-t5-small' 
# #default_config["method"]["gen_kwargs"]['bos_token'] = '<s>'
# #default_config['train']['gradient_accumulation_steps']=2
# config = TRLConfig.update(default_config, {})
# print(config)

In [ ]:
#os.environ['WANDB_SILENT']="true"

# google/flan-t5-large + ILQL, no LORA

用FLAN-T5试试 ，不用gpt2 ，注意config的定义方法

In [ ]:
config = TRLConfig(
    train=TrainConfig(
        seq_length=256,
        epochs=100,
        total_steps=1000,
        batch_size=8,
        checkpoint_interval=100,
        eval_interval=10,
        pipeline="PromptPipeline",
        trainer="AccelerateILQLTrainer",#"AcceleratePPOTrainer", #
        tracker=None, #default is wandb
    ),
    model=ModelConfig(
        model_path="google/flan-t5-large",
        model_arch_type="seq2seq",
        num_layers_unfrozen=2,
    ),
    tokenizer=TokenizerConfig(
        tokenizer_path="google/flan-t5-large",
        truncation_side="right",
    ),
    optimizer=OptimizerConfig(
        name="adamw",
        kwargs={
            "lr": 1.0e-5,
            "betas": [0.9, 0.999],
            "eps": 1.0e-8,
            "weight_decay": 1.0e-6,
        },
    ),
    scheduler=SchedulerConfig(
        name="cosine_annealing",
        kwargs={
            "T_max": 10000,
            "eta_min": 1.0e-6,
        },
    ),
    method=ILQLConfig(name='ilqlconfig', 
               tau=0.7, 
               gamma=0.99, 
               cql_scale=0.1, 
               awac_scale=1, alpha=0.001, 
               beta=0, 
               steps_for_target_q_sync=5, 
               two_qs =True,
               gen_kwargs={'max_new_tokens': 56, 
                    'top_k': 20, 
                    'beta': 1,
                     'temperature': 1.0}
        ),
    # method =ILQLConfig(
        
    #     name ="ILQLConfig",
    #     tau =  0.7,
    #     gamma = 0.99,
    #     cql_scale = 0.1,
    #     awac_scale =  1,
    #     alpha =  0.001,
    #     beta =  0,
    #     steps_for_target_q_sync =  5,
    #     two_qs = True,
    #     gen_kwargs={
            
    #         "max_new_tokens": 56,
    #         "top_k": 20,
    #         "beta": 1,
    #         "temperature": 1.0
    #     }
    # ),
)

In [ ]:
#这里解释一下 get_positive_score(scores) 的作用方式
scores = [
    { "SENTIMENT":"POSITIVE", "score": 0.8},
   { "SENTIMENT":"NEGETIVE", "score": 0.2},
    #{ "SENTIMENT":"NEUTRAL", "score": 0.1},
]
# Apply the map() and dict() functions to extract the value associated with the "POSITIVE" key
'''
这里map函数的用法: map(func ,list[或者其他iterable比如tuple，dict])
func这里的入参是list中的元素而不是list本身
func这里具体的实现是一个lambda函数 函数的入参x是list中的一个字典结构元素，比如{ "SENTIMENT":"POSITIVE", "score": 0.8}
lambda函数将这个字典结构（本身是{k1:v1 , k2:v2}）转换成元组，因为只提取了每个x.values(),所以元组中的值只包含v1和v2
即各个tuple组成了一个list 为 [('POSITIVE',0.8),("NEGATIVE",-0.4)]
然后又使用dict函数将该list转为字典{'POSITIVE':0.8,"NEGATIVE":-0.4}
可以测试这段代码就明白
>>> a=[(1,2),(3,4)]
>>> a
[(1, 2), (3, 4)]
>>> dict(a)
{1: 2, 3: 4}
最后只取字典中"POSITIVE"对应的值作为最终的评分

结合下面代码
trainer = trlx.train(
    samples=imdb["text"], 
    rewards=imdb["label"],
    ....
来看 这个rewards=imdb['label']的取值只有0和1 ，0代表负面，1代表正面，这个reward的定义方式就是鼓励输出正面，因为这样奖励值大（1>0）
正好又能代表奖励，即输出负面文字奖励值低（为0），输出正面文字奖励值高（1）
而positive_score = dict(map(lambda x: tuple(x.values()), scores))["POSITIVE"]这个值是由distiBERT模型评价transfoermer模型输出文字的得分
其实这个得分就是归一化的分布概率，就是softmax最后输出的那个玩意 想起来没，就是logits，所以加起来是1。这里只不过用到了带温度的softmax，本质一样都是logits。
这个得分要越高越好，而这正是ILQL或者PPO算法通过得分positive score值返过去调整transformer模型参数来实现的。
'''

positive_score = dict(map(lambda x: tuple(x.values()), scores))["POSITIVE"]

#每个x是{ "SENTIMENT":"POSITIVE", "score": 0.8} x.values()是两个值，tuple之后是("POSITIVE",0.8),dict就是{"POSITIVE":0.8}
#把所有的x都考虑上 dict最后就是{"POSITIVE":0.8  ,"NEGATIVE":-0.4 ,"NEUTRAL":0.2}
print(dict(map(lambda x: tuple(x.values()), scores)))
# Print the value associated with the "POSITIVE" key 这步就是取出其中的POSITIVE对应的值当作奖励值
print(positive_score)

{'POSITIVE': 0.8, 'NEGETIVE': 0.2}
0.8


In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
print(device)

cpu


In [ ]:
def get_positive_score(scores):
    "Extract value associated with a positive sentiment from pipeline's output"
    return dict(map(lambda x: tuple(x.values()), scores))["NEGATIVE"]#["POSITIVE"]  #return的是一个float

#pipeline huggingface提供的一种使用模型的快捷方式 参考[https://huggingface.co/docs/transformers/main_classes/pipelines]
'''#例子
pipe = pipeline(model="roberta-large-mnli")
pipe("This restaurant is awesome")
[{'label': 'NEUTRAL', 'score': 0.7313136458396912}]
'''
sentiment_fn = pipeline(
     
    "sentiment-analysis",
    "lvwerra/distilbert-imdb",
    top_k=2, #只输出POSTIVE和NEGETIVE两类的归一化概率logits
    truncation=True,
    batch_size=6,
    device=torch.device("cuda" if torch.cuda.is_available() else "cpu")#0,
)

def sentiment_fn_trick_test_20230416(samples):
   lens =len(samples)
   return [[{'label': 'NEGATIVE', 'score': 0.76}, {'label': 'POSITIVE', 'score': 0.24}] for _ in range(lens)]
'''
说明一下metric_fn的作用
samples=[影评1，影评2]
sentiment_fn(samples)会对每个影评做一个正负情感分类，参考distilbert-imdb例子[https://huggingface.co/lvwerra/distilbert-imdb]
sentiment_fn(samples) 是一个list= [{'label': 'NEGATIVE', 'score': 0.76}, {'label': 'POSITIVE', 'score': 0.24}]
而map(get_positive_score, sentiment_fn(samples)) 将这个list中的每个dict转化成一个得分值score，再list一下，成为[影评1得分，影评2得分]的score_list
最后return {'sentiments':score_list} 我觉得这个字典的key是任意写的
'''

def metric_fn(samples: List[str], **kwargs) -> Dict[str, List[float]]:
    sentiments = list(map(get_positive_score, sentiment_fn(samples)))
    return {"sentiments": sentiments}

imdb = load_dataset("imdb", split="train+test")

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0. Subsequent calls will reuse this data.


In [ ]:
#%cd /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0

In [ ]:
#%ls

In [ ]:
#原始的imdb可能结构就是个jsonl 有50000行，每行形如{'text':XXX , 'label':0 or 1}
#然后被转成了datasets.arrow_dataset.Dataset 结构，形如{'text':[t1,t2,t3,...] ,'label':[0,1,0,...] }
#这跟之前我尝试自定义的结构是一致的

In [ ]:
#这是一种定义数据的方式 类似csv 每个key都把所有的值存成list 这个a里面是嵌套的json
import json
q=[1,2]
p=[11,22]
a={'data_filed':{'q':q ,'p':p}}
with open('a','w') as f:
  json.dump(a,f)  #需要序列化 是直接把整个obj序列化到文件
#load时 由于a是嵌套json 所以需要加入filed=XX指明哪个字段
a_load = load_dataset("json", data_files="a",field='data_filed')
print(a_load)
print(a_load['train']['q'])
print(a_load['train'][:2])

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-5f101f42f7729c1f/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['q', 'p'],
        num_rows: 2
    })
})
[1, 2]
{'q': [1, 2], 'p': [11, 22]}


In [ ]:
#这是另一种定义数据的方式 json或者叫jsonline 每行定义一个 1个key存1个值 不是a那样的嵌套json
import json
mydata = [
    {'q':1 ,'p':11} , 
    {'q':2 ,'p':22}
]

with open('b','w') as f:
  for item in mydata:
    #需要序列化 是把每一个dict对象序列化再写入文件
    json_line = json.dumps(item, ensure_ascii=False)
    f.write(json_line + "\n")


b_load = load_dataset("json", data_files="b")
print(b_load)
print(b_load['train']['q'])
print(b_load['train'][:2])

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-847c29d6df4d15bb/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['q', 'p'],
        num_rows: 2
    })
})
[1, 2]
{'q': [1, 2], 'p': [11, 22]}


In [ ]:
#测试一下
print(type(imdb))
#这种datasets.arrow_dataset.Dataset 类型的数据
#可以按照key取值
print(imdb['label'][:2])
print(imdb['text'][:2])
#也可以把所有的key对应的value list同时取相同的长度或者具体某一条sample
print(imdb[:2]) #这是嵌套json {'text':[t1,t2] , 'label':[l1,l2]}
print(imdb[2])  #这是jsonl结构 ｛'text':xXXX, 'label':YYY｝

<class 'datasets.arrow_dataset.Dataset'>
[0, 0]
['I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the 

In [ ]:
'''
说明一下metric_fn的作用
samples=[影评1，影评2]
sentiment_fn(samples)会对每个影评做一个正负情感分类，参考distilbert-imdb例子[https://huggingface.co/lvwerra/distilbert-imdb]
sentiment_fn(samples) 是一个嵌套list 与上面举例一致 除了key值不同但是这个不影响 因为我们只要values 
sentiment_fn(samples)结果形如 
[
  [{'label': 'NEGATIVE', 'score': 0.7600291967391968}, {'label': 'POSITIVE', 'score': 0.23997080326080322}],
  [{'label': 'NEGATIVE', 'score': 0.9923253059387207}, {'label': 'POSITIVE', 'score': 0.00767475226894021}],
  [{'label': 'NEGATIVE', 'score': 0.9928519129753113}, {'label': 'POSITIVE', 'score': 0.007148080505430698}]
]
而map(get_positive_score, sentiment_fn(samples)) 将这个list中的内部list转化成一个得分值score，再list一下，成为[影评1得分，影评2得分]的score_list
最后return {'sentiments':score_list} 我觉得这个字典的key是任意写的
'''
#注意是imdb['text']不是imdb,因为imdb包含text和label两个key 这里只能输入text对应的list
small_samples = imdb[:3]["text"]
print(sentiment_fn(small_samples))
#get_postive_score（函数定义内部就是map方式作用）的输入是一个内部[{'label': 'NEGATIVE', 'score': 0.7600291967391968}, {'label': 'POSITIVE', 'score': 0.23997080326080322}]
#得到一个得分0.23997080326080322
#map(get_postive_score, XXX) 就是把XXX中每个list转化为一个得分，外面加上list形成列表
print(list(map(get_positive_score, sentiment_fn(small_samples))))

[[{'label': 'NEGATIVE', 'score': 0.7600291967391968}, {'label': 'POSITIVE', 'score': 0.23997080326080322}], [{'label': 'NEGATIVE', 'score': 0.9923253059387207}, {'label': 'POSITIVE', 'score': 0.00767475226894021}], [{'label': 'NEGATIVE', 'score': 0.9928519129753113}, {'label': 'POSITIVE', 'score': 0.007148080505430698}]]
[0.7600291967391968, 0.9923253059387207, 0.9928519129753113]


In [ ]:
#先看看训练的内容是什么
print(f"len_of_training_samples={len(imdb)}")
print(imdb[10000])
count =0

#label=0 negetive
#label=1 positive

for item in imdb:
  if item['label'] ==0 :  #0表示负面影评 我们这里只是看一下 后面要用的实际是label=1的正面影评
    print(item['text'])
    print(item['label'])
    count +=1
  if count ==10:break

len_of_training_samples=50000
{'text': "Someone actually gave this movie 2 stars. There's a very high chance they need immediate professional help as anyone who doesn't spend 30 seconds to see if you can award no stars is quite literally scary.<br /><br />This film is ... well ... I guess it's pretty much some kind of attempt at a horrible porn / snuff movie with no porn or no real horrible bits (apart from the acting, plot, story, sets, dialogue and sound). I wrongly assumed it was about zombies. <br /><br />Watching it is actually quite scary in fairness; you're terrified someone will come over and you'll never be able to describe what it is and they'll go away thinking you're a freak that watches home-made amateur torture videos or something along those lines. <br /><br />I'm so taken aback I'm writing this review on my mobile so I don't forget to attempt to bring the rating down further than the current 1.6 to save others from the same horrible fate that I just suffered. <br /><br 

In [ ]:
#显存太小 减少训练样本
#train_num=20000

In [ ]:
#参考 https://www.kaggle.com/code/shiyan00221/ppo-aidev-2023
#   build_dataset_test 方法中tokenize(sample)函数加入了add_special_tokens=True 
#   然后把所有的sample处理成input_ids 又反过来处理成文字形式 这多此一举的行为肯定有道理
#add_special_tokens=False or True 都试过了 没用 还是报 bos_toekn is used but not set
tokenizer = AutoTokenizer.from_pretrained(config.model.model_path)
def tokenize(sample):
    sample["input_ids"] = tokenizer.encode(sample["text"], add_special_tokens=False)
    sample["query"] = tokenizer.decode(sample["input_ids"])
    return sample

参考dataset的各种预处理 对列的变换

https://huggingface.co/docs/datasets/process

In [ ]:
#不用执行 只是看看remove_colunms函数的用法
imdb = load_dataset("imdb", split="train+test")
print(imdb) #此时imdb有2列 text和label
imdb_2 = imdb.remove_columns(['label']) # 可以用这个方法去掉不需要的
print(imdb_2)

Dataset({
    features: ['text', 'label'],
    num_rows: 50000
})
Dataset({
    features: ['text'],
    num_rows: 50000
})


In [ ]:
#不用执行
#没啥用 还是会大量报Using bos_token, but it is not set yet.
#Using bos_token, but it is not set yet.
#Using bos_token, but it is not set yet.
ds = imdb
ds = ds.map(tokenize, batched=False)
ds.set_format(type='torch')
ds = ds.remove_columns(['text','input_ids'])

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
#print(ds)

Dataset({
    features: ['label', 'query'],
    num_rows: 50000
})


In [ ]:
#trainer中的eval_prompts是用于在训练过程中验证效果的样本
#看看用这些提示输入，模型返回的句子是怎么样的 ，理论上输出会根据这些提示输出正面评价的output
#Evaluation #11 metrics/sentiments: 0.528 这个值是句子的正面分类得分 由上面metric_fn的定义而来


trainer = trlx.train(
    
    #prompts=prompts,
    #eval_prompts=val_prompts,
    #reward_fn=metric_fn,
    #config=config,
    samples=imdb["text"][:20000] ,#ds['query'][:200], #imdb["text"][:20000],  #输入文本
    #注意这个rewards和scores函数对应 做减法就是scores函数取NEGATIVE的值
    #rewards=[ 1.0 - value  for value in ds["label"][:200]],
    rewards=[ 1.0 - value  for value in imdb["label"][:20000]]
    #,  #被标注的label=0 ，1 代表负面或者正面，既然是reward那么越高越高 所以这是引导输出1即正面文字
    eval_prompts=[  #训练过程中的验证 验证时会看到输出根据训练的进行在逐步改变成正面文字
        "I don't know much about Hungarian underground",
        "What made this movie so distinctly",
        "Like the sandwich I just bought at the grocery store,",
        "I cannot believe how much this movie made me want to"
    ] *1, #* 20,
    metric_fn=metric_fn, #这个metric_fn会把上面的samples作为输入
    config=config,
)

[RANK 0] Initializing model: google/flan-t5-large
[RANK 0] Collecting rollouts
Using bos_token, but it is not set yet.
Using bos_token, but it is not set yet.
Using bos_token, but it is not set yet.
Using bos_token, but it is not set yet.
Using bos_token, but it is not set yet.
Token indices sequence length is longer than the specified maximum sequence length for this model (576 > 512). Running this sequence through the model will result in indexing errors
Using bos_token, but it is not set yet.
Using bos_token, but it is not set yet.
Using bos_token, but it is not set yet.
Using bos_token, but it is not set yet.
Using bos_token, but it is not set yet.
Using bos_token, but it is not set yet.
Using bos_token, but it is not set yet.
Using bos_token, but it is not set yet.
Using bos_token, but it is not set yet.
Using bos_token, but it is not set yet.
Using bos_token, but it is not set yet.
Using bos_token, but it is not set yet.
Using bos_token, but it is not set yet.
Using bos_token, bu

                                                  Sample Example                                                   
┏━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Prompt ┃ Response                                                                                  ┃ Reward     ┃
┡━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ </s>   │ I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that      │ tensor(1.) │
│        │ surrounded it when it was first released in 1967. I also heard that at first it was       │            │
│        │ seized by U.S. customs if it ever tried to enter this country, therefore being a fan of   │            │
│        │ films considered "controversial" I really had to see this for myself.<unk> br /><unk> br  │            │
│        │ />The plot is centered around a young Swedish drama student named Lena who wants to learn │            │
│        │ everything she can about life. In particular she wants to focus her attentions to making  │            │
│        │ some sort of documentary on what the average Swede thought about certain political issues │            │
│        │ such as the Vietnam War and race issues in the United States. In between asking           │            │
│        │ politicians and ordinary denizens of Stockholm about their opinions on politics, she has  │            │
│        │ sex with her drama teacher, classmates, and married men.<unk> br /><unk> br />What kills  │            │
│        │ me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic.      │            │
│        │ Really, the sex and nudity scenes are few and far between, even then it's not shot like   │            │
└────────┴───────────────────────────────────────────────────────────────────────────────────────────┴────────────┘

[RANK 0] Logging experience string statistics


        Experience String Stats (mean ∈ [min, max])        
┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┓
┃ Prompt Length ┃ Output Length      ┃ Sample Length      ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━┩
│ 1.00 ∈ [1, 1] │ 216.04 ∈ [51, 255] │ 217.04 ∈ [52, 256] │
└───────────────┴────────────────────┴────────────────────┘

[RANK 0] Starting training
[RANK 0] Evaluating model


[generation sweep 0/1 | eval batch 0/1]:   0%|          | 0/1 [00:00<?, ?it/s]

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 6>:6                                                                              │
│                                                                                                  │
│ /content/drive/MyDrive/trlx_0403/trlx/trlx/trlx.py:124 in train                                  │
│                                                                                                  │
│   121 │   eval_pipeline = get_pipeline(config.train.pipeline)(eval_prompts, max_prompt_length,   │
│   122 │   trainer.add_eval_pipeline(eval_pipeline)                                               │
│   123 │                                                                                          │
│ ❱ 124 │   trainer.learn()                                                                        │
│   125 │   return trainer                                                                         │
│   126                                                                                            │
│                                                                                                  │
│ /content/drive/MyDrive/trlx_0403/trlx/trlx/trainer/accelerate_base_trainer.py:504 in learn       │
│                                                                                                  │
│   501 │   │   │   │   │   │   state = json.load(f)                                               │
│   502 │   │   │   │   │   │   self.iter_count = state["iter_count"]                              │
│   503 │   │   else:                                                                              │
│ ❱ 504 │   │   │   results = self.evaluate()                                                      │
│   505 │   │   │   self.accelerator.log(results, step=self.iter_count)                            │
│   506 │   │                                                                                      │
│   507 │   │   tbar = logging.tqdm(                                                               │
│                                                                                                  │
│ /content/drive/MyDrive/trlx_0403/trlx/trlx/trainer/accelerate_base_trainer.py:349 in evaluate    │
│                                                                                                  │
│   346 │   │   │   │   │   │   prompts["input_ids"], prompts["attention_mask"], **{gen_sweep_ar   │
│   347 │   │   │   │   │   )                                                                      │
│   348 │   │   │   │   else:                                                                      │
│ ❱ 349 │   │   │   │   │   samples = self.generate_eval(prompts["input_ids"], prompts["attentio   │
│   350 │   │   │   │                                                                              │
│   351 │   │   │   │   # TODO(reciprocated): this should be moved into `decode`                   │
│   352 │   │   │   │   # but that needs to be synced with indexing in `make_experience`           │
│                                                                                                  │
│ /content/drive/MyDrive/trlx_0403/trlx/trlx/trainer/accelerate_base_trainer.py:263 in             │
│ generate_eval                                                                                    │
│                                                                                                  │
│   260 │   │   kwargs = dict(self.generate_kwargs, **kwargs)                                      │
│   261 │   │                                                                                      │
│   262 │   │   with torch.no_grad():                                                              │
│ ❱ 263 │   │   │   return self.accelerator.unwrap_model(self.model).generate(                     │
│   264 │   │   │   │   input_ids=input_ids, attention_mask=a

上面是磁盘满了导致的

官网


https://huggingface.co/blog/rlhf

https://wandb.ai/carperai/summarize_RLHF/reports/Implementing-RLHF-Learning-to-Summarize-with-trlX--VmlldzozMzAwODM2

https://blog.eleuther.ai/trlx-exploratory-analysis/

模型训练完成后存储 网上说有两种方式
Important: Once the model is trained, you will need to save it in a particular way before you can load it into TransformerLens. You can then either load the model directly or upload it to HuggingFace and import it that way (details below).

Weights&Bais官网说是
trainer.save('/path/to/output/folder/')

下面是另外的说法来自https://blog.eleuther.ai/trlx-exploratory-analysis/
trainer.model.base_model.save_pretrained("base_model/")

In [ ]:
# output
input_str = 'One thing you should know about When Sally Met Harry is that'
trainer_output = trainer.generate_eval(
    **trainer.tokenizer(input_str, return_tensors='pt'))[0]
print(trainer.tokenizer.decode(trainer_output))

One thing you should know about When Sally Met Harry is that her name is the first thing you hear in a TV movie about "The Man In The White Suit." I've seen all of the films so far and it's not a bad film for sure. It does have some of the best writing and acting ever in


In [ ]:
trlx 
trl
gpt2：1.y
flan-t5 5-6y

# 20230417 引入LORA:  google/flan-t5-large + lora


In [ ]:
config = TRLConfig(
    train=TrainConfig(
        seq_length=256, 
        #真正的train step = max(epochs , total_steps) 
        #参见/content/drive/MyDrive/trlx_0403/trlx/configs/test_config.yml 的定义
        epochs=100,
        total_steps=5000,
        batch_size=8,
        checkpoint_interval=5000, #训练中每checkpoint_interval次就保存一次 但是colab只有15G 所以我们最后再保存
        eval_interval=500,
        pipeline="PromptPipeline",
        trainer="AccelerateILQLTrainer",#"AcceleratePPOTrainer", #
        tracker=None, #default is wandb
    ),
    model=ModelConfig(
        model_path='google/flan-t5-large',#'google/flan-t5-large',#"gpt2",
        model_arch_type="seq2seq",      #causal for gpt2
        num_layers_unfrozen=2,
        #load_in_8bit =True,  not supoort yet
        # LORA here 
        delta_kwargs={
                "delta_type": "lora", 
                #gpt2不适用这个"modified_modules": "attention",
                'modified_modules':'attention', # all
                "lora_r": 8, 
                "lora_alpha": 32, 
                "lora_dropout": 0.05,
                #"task_type":"CAUSAL_LM",  not supoort yet
                },
                 
    ),
    tokenizer=TokenizerConfig(
        tokenizer_path='google/flan-t5-large',#'google/flan-t5-large',#"gpt2",
        truncation_side="right",
    ),
    optimizer=OptimizerConfig(
        name="adamw",
        kwargs={
            "lr": 1.0e-5,
            "betas": [0.9, 0.999],
            "eps": 1.0e-8,
            "weight_decay": 1.0e-6,
        },
    ),
    scheduler=SchedulerConfig(
        name="cosine_annealing",
        kwargs={
            "T_max": 10000,
            "eta_min": 1.0e-6,
        },
    ),
    method=ILQLConfig(name='ilqlconfig', 
               tau=0.7, 
               gamma=0.99, 
               cql_scale=0.1, 
               awac_scale=1, alpha=0.001, 
               beta=0, 
               steps_for_target_q_sync=5, 
               two_qs =True,
               gen_kwargs={'max_new_tokens': 56, 
                    'top_k': 20, 
                    'beta': 1,
                     'temperature': 1.0}
        ),
    # method =ILQLConfig(
        
    #     name ="ILQLConfig",
    #     tau =  0.7,
    #     gamma = 0.99,
    #     cql_scale = 0.1,
    #     awac_scale =  1,
    #     alpha =  0.001,
    #     beta =  0,
    #     steps_for_target_q_sync =  5,
    #     two_qs = True,
    #     gen_kwargs={
            
    #         "max_new_tokens": 56,
    #         "top_k": 20,
    #         "beta": 1,
    #         "temperature": 1.0
    #     }
    # ),
)

## 试试LoRA更多 不做UNfrozen限制  并且lora:attention换成Lora:all

In [ ]:
lora:atten 注意力层 * 24
low rank matrice:rank
model.bin +lora_adapter.bin: output

In [ ]:
config = TRLConfig(
    train=TrainConfig(
        seq_length=256, 
        #真正的train step = max(epochs , total_steps) 
        #参见/content/drive/MyDrive/trlx_0403/trlx/configs/test_config.yml 的定义
        epochs=100,
        total_steps=5000,
        batch_size=8,
        checkpoint_interval=5000, #训练中每checkpoint_interval次就保存一次 但是colab只有15G 所以我们最后再保存
        eval_interval=500,
        pipeline="PromptPipeline",
        trainer="AccelerateILQLTrainer",#"AcceleratePPOTrainer", #
        tracker=None, #default is wandb
    ),
    model=ModelConfig(
        model_path='google/flan-t5-large',#'google/flan-t5-large',#"gpt2",
        model_arch_type="seq2seq",      #causal for gpt2
        #num_layers_unfrozen=2,
        #load_in_8bit =True,  not supoort yet
        #LORA here 
        delta_kwargs={
                "delta_type": "lora", 
                #gpt2不适用这个"modified_modules": "attention",
                'modified_modules':"all",#'attention', # all
                "lora_r": 8, 
                "lora_alpha": 32, 
                "lora_dropout": 0.05,
                #"task_type":"CAUSAL_LM",  not supoort yet
                },
                 
    ),
    tokenizer=TokenizerConfig(
        tokenizer_path='google/flan-t5-large',#'google/flan-t5-large',#"gpt2",
        truncation_side="right",
    ),
    optimizer=OptimizerConfig(
        name="adamw",
        kwargs={
            "lr": 1.0e-5,
            "betas": [0.9, 0.999],
            "eps": 1.0e-8,
            "weight_decay": 1.0e-6,
        },
    ),
    scheduler=SchedulerConfig(
        name="cosine_annealing",
        kwargs={
            "T_max": 10000,
            "eta_min": 1.0e-6,
        },
    ),
    method=ILQLConfig(name='ilqlconfig', 
               tau=0.7, 
               gamma=0.99, 
               cql_scale=0.1, 
               awac_scale=1, alpha=0.001, 
               beta=0, 
               steps_for_target_q_sync=5, 
               two_qs =True,
               gen_kwargs={'max_new_tokens': 56, 
                    'top_k': 20, 
                    'beta': 1,
                     'temperature': 1.0}
        ),
    # method =ILQLConfig(
        
    #     name ="ILQLConfig",
    #     tau =  0.7,
    #     gamma = 0.99,
    #     cql_scale = 0.1,
    #     awac_scale =  1,
    #     alpha =  0.001,
    #     beta =  0,
    #     steps_for_target_q_sync =  5,
    #     two_qs = True,
    #     gen_kwargs={
            
    #         "max_new_tokens": 56,
    #         "top_k": 20,
    #         "beta": 1,
    #         "temperature": 1.0
    #     }
    # ),
)

In [ ]:
import torch

In [ ]:
#torch.cuda.empty_cache()

In [ ]:
def get_positive_score(scores):
    "Extract value associated with a positive sentiment from pipeline's output"
    return dict(map(lambda x: tuple(x.values()), scores))["NEGATIVE"]#["POSITIVE"]  #return的是一个float

#pipeline huggingface提供的一种使用模型的快捷方式 参考[https://huggingface.co/docs/transformers/main_classes/pipelines]
'''#例子
pipe = pipeline(model="roberta-large-mnli")
pipe("This restaurant is awesome")
[{'label': 'NEUTRAL', 'score': 0.7313136458396912}]
'''
sentiment_fn = pipeline( #RM hf
     
    "sentiment-analysis",
    "lvwerra/distilbert-imdb",
    top_k=2, #只输出POSTIVE和NEGETIVE两类的归一化概率logits
    truncation=True,
    batch_size=8,
    #device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")#0,
)

def sentiment_fn_trick_test_20230416(samples):
   lens =len(samples)
   return [[{'label': 'NEGATIVE', 'score': 0.76}, {'label': 'POSITIVE', 'score': 0.24}] for _ in range(lens)]
'''
说明一下metric_fn的作用
samples=[影评1，影评2]
sentiment_fn(samples)会对每个影评做一个正负情感分类，参考distilbert-imdb例子[https://huggingface.co/lvwerra/distilbert-imdb]
sentiment_fn(samples) 是一个list= [{'label': 'NEGATIVE', 'score': 0.76}, {'label': 'POSITIVE', 'score': 0.24}]
而map(get_positive_score, sentiment_fn(samples)) 将这个list中的每个dict转化成一个得分值score，再list一下，成为[影评1得分，影评2得分]的score_list
最后return {'sentiments':score_list} 我觉得这个字典的key是任意写的
'''

def metric_fn(samples: List[str], **kwargs) -> Dict[str, List[float]]:
    sentiments = list(map(get_positive_score, sentiment_fn(samples)))
    return {"sentiments": sentiments}

imdb = load_dataset("imdb", split="train+test")

In [ ]:
#构建一个较小的数据集 以jsonl的形式
imdb_positive_500 = []
len_pos =0
imdb_negative_500 = []
len_neg =0
for i in range(len(imdb)):
  label = imdb[i]['label']
  if label == 1 and len_pos<500:
    pos_review = imdb[i]['text']
    imdb_positive_500.append({'text':pos_review , 'label':label})
    len_pos += 1
  if label == 0 and len_neg<500:
    neg_review = imdb[i]['text']
    imdb_negative_500.append({'text':neg_review , 'label':label})
    len_neg += 1

In [ ]:
#imdb_negative_500

In [ ]:
import random
imdb_1k = imdb_positive_500 + imdb_negative_500


random.shuffle(imdb_1k) #随机打乱
len(imdb_1k)

1000

In [ ]:
import json
with open('/content/drive/MyDrive/trlx_0403/imdb_1k.json','w') as f:
  for item in imdb_1k :
    f.write(json.dumps(item))
    f.write('\n')
print('json write done')

json write done


In [ ]:
from datasets import load_dataset
#带上split='train' 这样导入后就没有train这一层 结构更简单 不然要用#imdb_1k_ds['train']['text'][:4] 多一层参数
imdb_1k_ds = load_dataset("json", data_files="/content/drive/MyDrive/trlx_0403/imdb_1k.json",split="train")

print(imdb_1k_ds)


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-66411290daf6a649/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e. Subsequent calls will reuse this data.
Dataset({
    features: ['text', 'label'],
    num_rows: 1000
})


In [ ]:
#imdb
#imdb_1k_ds 
#目前接口一致

In [ ]:
print([  1.0 -value  for value in imdb_1k_ds["label"]][0])
print(imdb_1k_ds["text"][0])

1.0
Who are these "They"- the actors? the filmmakers? Certainly couldn't be the audience- this is among the most air-puffed productions in existence. It's the kind of movie that looks like it was a lot of fun to shoot TOO much fun, nobody is getting any actual work done, and that almost always makes for a movie that's no fun to watch.<br /><br />Ritter dons glasses so as to hammer home his character's status as a sort of doppleganger of the bespectacled Bogdanovich; the scenes with the breezy Ms. Stratten are sweet, but have an embarrassing, look-guys-I'm-dating-the-prom-queen feel to them. Ben Gazzara sports his usual cat's-got-canary grin in a futile attempt to elevate the meager plot, which requires him to pursue Audrey Hepburn with all the interest of a narcoleptic at an insomnia clinic. In the meantime, the budding couple's respective children (nepotism alert: Bogdanovich's daughters) spew cute and pick up some fairly disturbing pointers on 'love' while observing their parents. (

3145728/786295808 = 0.40%     Lora：atten

5357568/788507648 = 0.68%     Lora：all

明天减小batch size测试


In [ ]:
MLP=FCN

In [ ]:

#trainer中的eval_prompts是用于在训练过程中验证效果的样本
#看看用这些提示输入，模型返回的句子是怎么样的 ，理论上输出会根据这些提示输出正面评价的output
#Evaluation #11 metrics/sentiments: 0.528 这个值是句子的正面分类得分 由上面metric_fn的定义而来


trainer = trlx.train(
    
    #prompts=prompts,
    #eval_prompts=val_prompts,
    #reward_fn=metric_fn,
    #config=config,
    samples=imdb_1k_ds['text'],#imdb["text"][:1000] ,#ds['query'][:200], #imdb["text"][:20000],  #输入文本
    #注意这个rewards和scores函数对应 做减法就是scores函数取NEGATIVE的值
    #rewards=[ 1.0 - value  for value in ds["label"][:200]],
    rewards=[  1.0 -value  for value in imdb_1k_ds["label"]],#[  1.0 -value  for value in imdb["label"][:1000]],
    #,  #被标注的label=0 ，1 代表负面或者正面，既然是reward那么越高越高 所以这是引导输出1即正面文字
    eval_prompts = [
        "What made this movie so",
        "This is the movie I will ",
        "This movie is such a ",
        "I didn't expect this movie was",
    ] * 8,
    metric_fn=metric_fn, #这个metric_fn会把上面的samples作为输入
    config=config,
)

[RANK 0] Initializing model: google/flan-t5-large


root
├── shared(Embedding),lm_head(Linear) weight:[32128, 1024]
├── encoder (T5Stack)
│   ├── embed_tokens (Embedding) weight:[32128, 1024]
│   ├── block (ModuleList)
│   │   ├── 0 (T5Block)
│   │   │   └── layer (ModuleList)
│   │   │       ├── 0 (T5LayerSelfAttention)
│   │   │       │   ├── SelfAttention (T5Attention)
│   │   │       │   │   ├── q,k,v,o(Linear) weight:[1024, 1024]
│   │   │       │   │   └── relative_attention_bias (Embedding) weight:[32, 16]
│   │   │       │   └── layer_norm (T5LayerNorm) weight:[1024]
│   │   │       └── 1 (T5LayerFF)
│   │   │           ├── DenseReluDense (T5DenseGatedActDense)
│   │   │           │   ├── wi_0,wi_1(Linear) weight:[2816, 1024]
│   │   │           │   └── wo (Linear) weight:[1024, 2816]
│   │   │           └── layer_norm (T5LayerNorm) weight:[1024]
│   │   └── 1-23(T5Block)
│   │       └── layer (ModuleList)
│   │           ├── 0 (T5LayerSelfAttention)
│   │           │   ├── SelfAttention (T5Attention)
│   │           │   │   └── q,k,v,o(Linear) weight:[1024, 1024]
│   │           │   └── layer_norm (T5LayerNorm) weight:[1024]
│   │           └── 1 (T5LayerFF)
│   │               ├── DenseReluDense (T5DenseGatedActDense)
│   │               │   ├── wi_0,wi_1(Linear) weight:[2816, 1024]
│   │               │   └── wo (Linear) weight:[1024, 2816]
│   │               └── layer_norm (T5LayerNorm) weight:[1024]
│   └── final_layer_norm (T5LayerNorm) weight:[1024]
└── decoder (T5Stack)
    ├── embed_tokens (Embedding) weight:[32128, 1024]
    ├── block (ModuleList)
    │   ├── 0 (T5Block)
    │   │   └── layer (ModuleList)
    │   │       ├── 0 (T5LayerSelfAttention)
    │   │       │   ├── SelfAttention (T5Attention)
    │   │       │   │   ├── q,k,v,o(Linear) weight:[1024, 1024]
    │   │       │   │   │   └── lora (LowRankLinear) lora_A:[8, 1024] lora_B:[1024, 8]
    │   │       │   │   └── relative_attention_bias (Embedding) weight:[32, 16]
    │   │       │   └── layer_norm (T5LayerNorm) weight:[1024]
    │   │       ├── 1 (T5LayerCrossAttention)
    │   │       │   ├── EncDecAttention (T5Attention)
    │   │       │   │   └── q,k,v,o(Linear) weight:[1024, 1024]
    │   │       │   │       └── lora (LowRankLinear) lora_A:[8, 1024] lora_B:[1024, 8]
    │   │       │   └── layer_norm (T5LayerNorm) weight:[1024]
    │   │       └── 2 (T5LayerFF)
    │   │           ├── DenseReluDense (T5DenseGatedActDense)
    │   │           │   ├── wi_0,wi_1(Linear) weight:[2816, 1024]
    │   │           │   │   └── lora (LowRankLinear) lora_A:[8, 1024] lora_B:[2816, 8]
    │   │           │   └── wo (Linear) weight:[1024, 2816]
    │   │           │       └── lora (LowRankLinear) lora_A:[8, 2816] lora_B:[1024, 8]
    │   │           └── layer_norm (T5LayerNorm) weight:[1024]
    │   └── 1-23(T5Block)
    │       └── layer (ModuleList)
    │           ├── 0 (T5LayerSelfAttention)
    │           │   ├── SelfAttention (T5Attention)
    │           │   │   └── q,k,v,o(Linear) weight:[1024, 1024]
    │           │   │       └── lora (LowRankLinear) lora_A:[8, 1024] lora_B:[1024, 8]
    │           │   └── layer_norm (T5LayerNorm) weight:[1024]
    │           ├── 1 (T5LayerCrossAttention)
    │           │   ├── EncDecAttention (T5Attention)
    │           │   │   └── q,k,v,o(Linear) weight:[1024, 1024]
    │           │   │       └── lora (LowRankLinear) lora_A:[8, 1024] lora_B:[1024, 8]
    │           │   └── layer_norm (T5LayerNorm) weight:[1024]
    │           └── 2 (T5LayerFF)
    │               ├── DenseReluDense (T5DenseGatedActDense)
    │               │   ├── wi_0,wi_1(Linear) weight:[2816, 1024]
    │               │   │   └── lora (LowRankLinear) lora_A:[8, 1024] lora_B:[2816, 8]
    │               │   └── wo (Linear) weight:[1024, 2816]
    │               │       └── lora (LowRankLinear) lora_A:[8, 2816] lora_B:[1024, 8]
    │               └── layer_norm (T5LayerNorm) weight:[1024]
    └── final_layer_norm (T5LayerNorm) weight:[1024]

[INFO|(OpenDelta)basemodel:696]2023-04-18 16:37:14,171 >> Trainable Ratio: 5357568/788507648=0.679457%
[INFO|(OpenDelta)basemodel:698]2023-04-18 16:37:14,173 >> Delta Parameter Ratio: 5357568/788507648=0.679457%
[INFO|(OpenDelta)basemodel:700]2023-04-18 16:37:14,176 >> Static Memory 0.00 GB, Max Memory 0.00 GB
[RANK 0] Collecting rollouts
Using bos_token, but it is not set yet.
Using bos_token, but it is not set yet.
Using bos_token, but it is not set yet.
Using bos_token, but it is not set yet.
Using bos_token, but it is not set yet.
Using bos_token, but it is not set yet.
Using bos_token, but it is not set yet.
Using bos_token, but it is not set yet.
Using bos_token, but it is not set yet.
Using bos_token, but it is not set yet.
Using bos_token, but it is not set yet.
Token indices sequence length is longer than the specified maximum sequence length for this model (608 > 512). Running this sequence through the model will result in indexing errors
Using bos_token, but it is not set ye

                                                  Sample Example                                                   
┏━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃ Prompt ┃ Response                                                                                      ┃ Reward ┃
┡━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ </s>   │ When Tsui Hark experiments, nothing and no one can withstand him. Legend of Zu is possibly    │ 0.0    │
│        │ 6Hours condensed into 1h40. One does not understand all, but like at "2001 A Space Odyssey"   │        │
│        │ you also don't have to, but one feels the power of the film to every second, every picture.   │        │
│        │ An extraordinary vision of the future of the 7th art and the one of the most pioneering,      │        │
│        │ astounding, rejoicing in the recent years. VITAL severe MASTERPIECE! It's absolutely perfect  │        │
│        │ as it is.<unk>br /><unk>br />When Tsui Hark experiments, nothing and no one can withstand     │        │
│        │ him. Legend of Zu is possibly 6Hours condensed into 1h40. One does not understand all, but    │        │
│        │ one feels the power of the film to every second, every picture. An extraordinary vision of    │        │
│        │ the future of the 7th art and the one of the most pioneering, astounding, rejoicing in the    │        │
│        │ recent years. VITAL severe MASTERPIECE! It's absolutely perfect as it is.                     │        │
│        │ 10000000000000/10000000000000</s>                                                             │        │
└────────┴───────────────────────────────────────────────────────────────────────────────────────────────┴────────┘

[RANK 0] Logging experience string statistics


...

[generation sweep 0/1 | eval batch 0/4]:   0%|          | 0/4 [00:00<?, ?it/s]

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
[RANK 0] Computing metrics
[RANK 0] Summarizing evaluation


                                      Evaluation #0 metrics/sentiments: 0.321                                      
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ prompt                       ┃ output                                                              ┃ sentiments ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ What made this movie so      │ This movie is just awful, the acting and editing was terrible, and  │ 0.997      │
│                              │ the storyline was just laugh out loud stupid. It would be a waste   │            │
│                              │ of money to even make an episode of this movie, what was with the   │            │
│                              │ original show?                                                      │            │
├──────────────────────────────┼─────────────────────────────────────────────────────────────────────┼────────────┤
│ This actor of this movie is  │ This actor of this movie this movie is this movie of actor of actor │ 0.286      │
├──────────────────────────────┼─────────────────────────────────────────────────────────────────────┼────────────┤
│ This plot of the movie is    │ This plot of the movie is the movie is the movie is the flir of the │ 0.218      │
│                              │ i n this plot the movie is the The movie is the the movie is the    │            │
│                              │ movie is this plot of the movie is the story about the asian sen    │            │
└──────────────────────────────┴─────────────────────────────────────────────────────────────────────┴────────────┘

  0%|          | 0/5000 [00:00<?, ?it/s]

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 6>:6                                                                              │
│                                                                                                  │
│ /content/drive/MyDrive/trlx_0403/trlx/trlx/trlx.py:124 in train                                  │
│                                                                                                  │
│   121 │   eval_pipeline = get_pipeline(config.train.pipeline)(eval_prompts, max_prompt_length,   │
│   122 │   trainer.add_eval_pipeline(eval_pipeline)                                               │
│   123 │                                                                                          │
│ ❱ 124 │   trainer.learn()                                                                        │
│   125 │   return trainer                                                                         │
│   126                                                                                            │
│                                                                                                  │
│ /content/drive/MyDrive/trlx_0403/trlx/trlx/trainer/accelerate_base_trainer.py:533 in learn       │
│                                                                                                  │
│   530 │   │   │   │   │   for mb in mbs:                                                         │
│   531 │   │   │   │   │   │   with self._accumulate():                                           │
│   532 │   │   │   │   │   │   │   forward_time -= time()                                         │
│ ❱ 533 │   │   │   │   │   │   │   loss, stats = self.loss(mb)                                    │
│   534 │   │   │   │   │   │   │   forward_time += time()                                         │
│   535 │   │   │   │   │   │   │   backward_time -= time()                                        │
│   536 │   │   │   │   │   │   │   self.accelerator.backward(loss)                                │
│                                                                                                  │
│ /content/drive/MyDrive/trlx_0403/trlx/trlx/trainer/accelerate_ilql_trainer.py:143 in loss        │
│                                                                                                  │
│   140 │   def loss(self, batch: Union[ILQLBatch, ILQLSeq2SeqBatch]):                             │
│   141 │   │   batch = to_device(batch, self.accelerator.device)                                  │
│   142 │   │   if self.config.model.model_arch_type == "seq2seq":                                 │
│ ❱ 143 │   │   │   logits, qs, target_qs, vs, _, _ = self.model(                                  │
│   144 │   │   │   │   input_ids=batch.input_ids,                                                 │
│   145 │   │   │   │   attention_mask=batch.attention_mask,                                       │
│   146 │   │   │   │   actions_ixs=batch.actions_ixs,                                             │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/torch/nn/modules/module.py:1501 in _call_impl             │
│                                                                                                  │
│   1498 │   │   if not (self._backward_hooks or self._backward_pre_hooks or self._forward_hooks   │
│   1499 │   │   │   │   or _global_backward_pre_hooks or _global_backward_hooks                   │
│   1500 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1501 │   │   │   return forward_call(*args, **kwargs)                                          │
│   1502 │   │   # Do not call functions when jit is used                                          │
│   1503 │   │   full_backward_hooks, non_full_backward_hooks

In [ ]:
# output
input_str = 'this plot of the movie is  '
trainer_output = trainer.generate_eval(
    **trainer.tokenizer(input_str, return_tensors='pt'))[0]
print(trainer.tokenizer.decode(trainer_output , skip_special_tokens=True))

This plot summary may be too long or excessively detailed. Please help improve it by removing unnecessary details and making it more concise. (May 2012) (Learn how and when to remove this template message) A SWAT team barges into a building that is


In [ ]:
# output
input_str_list = ['What do you think of the movie ?']*20
rlhf_t5_output_list = []
for input_str in input_str_list:
    trainer_output = trainer.generate_eval(**trainer.tokenizer(input_str, return_tensors='pt'))[0]
    rlhf_t5_output_list.append(trainer.tokenizer.decode(trainer_output , skip_special_tokens=True))
print(rlhf_t5_output_list)



['good', 'a tad bit predictable', 'good', "I think it's a very funny movie", 'good', 'Good but not great', 'good', 'okay', 'it was so bad', 'good', 'the cast and crew were so good that the whole movie was well worth seeing again', 'good', 'I thought that this movie is really good.', 'it was boring', 'good', 'okay', 'I like it a lot.', 'The movie was very funny and interesting.', 'It is a movie that can help you understand life better', 'The storyline is good.']


In [ ]:
rlhf_t5_output_list

['good',
 'a tad bit predictable',
 'good',
 "I think it's a very funny movie",
 'good',
 'Good but not great',
 'good',
 'okay',
 'it was so bad',
 'good',
 'the cast and crew were so good that the whole movie was well worth seeing again',
 'good',
 'I thought that this movie is really good.',
 'it was boring',
 'good',
 'okay',
 'I like it a lot.',
 'The movie was very funny and interesting.',
 'It is a movie that can help you understand life better',
 'The storyline is good.']

In [ ]:
metrics = metric_fn(rlhf_t5_output_list)
metrics

{'sentiments': [0.03137087821960449,
  0.9029200673103333,
  0.03137087821960449,
  0.007058396469801664,
  0.031370893120765686,
  0.780984103679657,
  0.03137087821960449,
  0.46163320541381836,
  0.9929181933403015,
  0.03137087821960449,
  0.009077776223421097,
  0.03137087821960449,
  0.015483071096241474,
  0.992356538772583,
  0.03137087821960449,
  0.46163320541381836,
  0.00731270294636488,
  0.009917120449244976,
  0.007533278316259384,
  0.019929053261876106]}

In [ ]:
3145728/786295808
#trainer中的eval_prompts是用于在训练过程中验证效果的样本
#看看用这些提示输入，模型返回的句子是怎么样的 ，理论上输出会根据这些提示输出正面评价的output
#Evaluation #11 metrics/sentiments: 0.528 这个值是句子的正面分类得分 由上面metric_fn的定义而来


trainer = trlx.train(
    
    #prompts=prompts,
    #eval_prompts=val_prompts,
    #reward_fn=metric_fn,
    #config=config,
    samples=imdb_1k_ds['text'],#imdb["text"][:1000] ,#ds['query'][:200], #imdb["text"][:20000],  #输入文本
    #注意这个rewards和scores函数对应 做减法就是scores函数取NEGATIVE的值
    #rewards=[ 1.0 - value  for value in ds["label"][:200]],
    rewards=[  1.0 -value  for value in imdb_1k_ds["label"]],#[  1.0 -value  for value in imdb["label"][:1000]],
    #,  #被标注的label=0 ，1 代表负面或者正面，既然是reward那么越高越高 所以这是引导输出1即正面文字
    eval_prompts = [
        "What made this movie so ",
        "This actor of this movie is  ",
        "This plot of the movie is ",
        "I didn't expect this movie was",
    ] * 8,
    metric_fn=metric_fn, #这个metric_fn会把上面的samples作为输入
    config=config,
)

[RANK 0] Initializing model: google/flan-t5-large


In [ ]:
imdb 50000 pos:25000 neg 25000


In [ ]:
''' 保存 save
Weights&Bais官网说是 
trainer.save('/path/to/output/folder/')

下面是另外的说法来自https://blog.eleuther.ai/trlx-exploratory-analysis/ 

trainer.model.base_model.save_pretrained("base_model/")
'''

In [ ]:
!pwd

/content/drive/MyDrive/trlx_0403/trlx


In [ ]:
trainer.save('/content/drive/MyDrive/20230418_flan_t5_large_lora/')

In [ ]:
#这个代码不对
trainer.model.base_model.save_pretrained("/content/drive/MyDrive/20230418_flan_t5_large_base_model/")

In [ ]:
#这才是保存基础模型到本地的正确代码
trainer.save_pretrained('/content/drive/MyDrive/20230418_flan_t5_large_pretrained_model/')

In [ ]:
# output
input_str = 'i think the movie is so   '
trainer_output = trainer.generate_eval(
    **trainer.tokenizer(input_str, return_tensors='pt'))[0]
print(trainer.tokenizer.decode(trainer_output))

# GPT2 + PPO 例子

In [ ]:
import torch,os
from typing import List
from datasets import load_dataset
from tqdm import tqdm
from transformers import AutoTokenizer

import trlx
from trlx.data.configs import (
    ModelConfig,
    OptimizerConfig,
    SchedulerConfig,
    TokenizerConfig,
    TrainConfig,
    TRLConfig,
)
from trlx.models.modeling_ppo import PPOConfig
from trlx.models.modeling_ilql import ILQLConfig

# try:
#     import evaluate
# except ImportError:
#     raise ImportError(
#         "To run this example, please install the `evaluate` and `nltk` packages" "by running `pip install evaluate`"
#     )

config = TRLConfig(
    train=TrainConfig(
        seq_length=256,
        epochs=5,
        total_steps=10,
        batch_size=8,
        checkpoint_interval=10,
        eval_interval=2,
        pipeline="PromptPipeline",
        trainer='AccelerateILQLTrainer',#"AcceleratePPOTrainer",
        tracker=None, #default is wandb
    ),
    model=ModelConfig(
        model_path="gpt2",
        model_arch_type="causul",
        num_layers_unfrozen=2,
    ),
    tokenizer=TokenizerConfig(
        tokenizer_path="gpt2",
        truncation_side="right",
    ),
    optimizer=OptimizerConfig(
        name="adamw",
        kwargs={
            "lr": 1.0e-5,
            "betas": [0.9, 0.999],
            "eps": 1.0e-8,
            "weight_decay": 1.0e-6,
        },
    ),
    scheduler=SchedulerConfig(
        name="cosine_annealing",
        kwargs={
            "T_max": 10000,
            "eta_min": 1.0e-6,
        },
    ),

    # method=PPOConfig(
    #     name="PPOConfig",
    #     num_rollouts=128,
    #     chunk_size=24,
    #     ppo_epochs=4,
    #     init_kl_coef=0.05,
    #     target=6,
    #     horizon=10000,
    #     gamma=0.99,
    #     lam=0.95,
    #     cliprange=0.2,
    #     cliprange_value=0.2,
    #     vf_coef=1.0,
    #     scale_reward=None,
    #     ref_mean=None,
    #     ref_std=None,
    #     cliprange_reward=10,
    #     gen_kwargs={
    #         "max_new_tokens": 100,
    #     },
    #     gen_experience_kwargs={
    #         "max_new_tokens": 60,
    #         "do_sample": True,
    #         "temperature": 1.0,
    #         "top_k": 5, #50,
    #         "top_p": 0.95,
    #     },
    # ),

    method =ILQLConfig(
        
        name ="ILQLConfig",
        tau =  0.7,
        gamma = 0.99,
        cql_scale = 0.1,
        awac_scale =  1,
        alpha =  0.001,
        beta =  0,
        steps_for_target_q_sync =  5,
        two_qs = True,
        gen_kwargs={
            
            "max_new_tokens": 56,
            "top_k": 20,
            "beta": 1,
            "temperature": 1.0
        }
    ),

)

╭──────────────────────────────────────────────────────────────────────────────────────────────────╮
│         epochs=,                                                                                 │
│                ▲                                                                                 │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
SyntaxError: invalid syntax

In [ ]:
#这个ppo_config.yml是哪儿来的
#default_config = yaml.safe_load(open("configs/ppo_config.yml"))

In [ ]:
%pwd

'/content/drive/MyDrive/trlx_0403/trlx'

In [ ]:
%ls /content/drive/MyDrive/trlx_0403/trlx/configs

accelerate/  nemo_configs/  sweeps/  test_config.yml


In [ ]:
#print(default_config)

In [ ]:
def get_negative_score(scores):
    "Extract value associated with a negative sentiment from pipeline's output"
    return dict(map(lambda x: tuple(x.values()), scores))["NEGATIVE"]

#default_config = yaml.safe_load(open("configs/ppo_config.yml"))

default_config = config

def main(hparams={}):
    config = TRLConfig.update(default_config, hparams)

    if torch.cuda.is_available():
        device = int(os.environ.get("LOCAL_RANK", 0))
    else:
        device = -1

    sentiment_fn = pipeline(
        "sentiment-analysis",
        "lvwerra/distilbert-imdb",
        top_k=2,
        truncation=True,
        batch_size=256,
        device=device,
    )

    def reward_fn(samples: List[str], **kwargs) -> List[float]:
        sentiments = list(map(get_negative_score, sentiment_fn(samples)))
        return sentiments

    # Take few words off of movies reviews as prompts
    imdb = load_dataset("imdb", split="train+test")
    prompts = [" ".join(review.split()[:4]) for review in imdb["text"]]
    print('see what the prompts look like:')
    print(prompts[:5])

    return trlx.train(
        #reward_fn=reward_fn,
        #prompts=prompts,
        metric_fn = reward_fn ,
        rewards = imdb['label'],
        samples = imdb['text'] ,
        eval_prompts=[#"It's hard to believe the sequel to Avatar has actually come out. After 13 years and what feels like half-a-dozen delays",
               #"This film seems to be boring." ,
               #"Do you recommend the movie ?",
               #"This movie makes me feel like"
               "What made this movie so ",
               "This movie was just like ",
               "This movie is not so ",
               "I didn't expect this movie was",
                      ] * 8,
        config=config,
    )

In [ ]:
trainer= main()

[RANK 0] Initializing model: gpt2


see what the prompts look like:
['I rented I AM', '"I Am Curious: Yellow"', 'If only to avoid', 'This film was probably', 'Oh, brother...after hearing about']


[RANK 0] Collecting rollouts
Token indices sequence length is longer than the specified maximum sequence length for this model (1169 > 1024). Running this sequence through the model will result in indexing errors


In [ ]:
# output
input_str = 'i think the movie is so'
trainer_output = trainer.generate_eval(
    **trainer.tokenizer(input_str, return_tensors='pt'))[0]
print(trainer.tokenizer.decode(trainer_output))

i think the movie is so much more than just a movie. It's a movie about a man who's lost his mind and his life. It's about a man who's lost his mind and his life.

It's about a man who's lost his mind and his life. It's about a man who's lost his mind and his life.

It's about a man who's lost his mind and his life. It's about a man who's lost his mind and his life.

It


In [ ]:
trainer.model.base_model.save_pretrained("/content/drive/MyDrive/ppo_trained_base_model/")

# 加载训练完的模型

In [ ]:
from transformers import pipeline,AutoTokenizer,AutoModelForCausalLM

In [ ]:
saved_model_path = "/content/drive/MyDrive/ppo_trained_base_model/"
rlhf_model = AutoModelForCausalLM.from_pretrained(saved_model_path)

In [ ]:
from transformers import GPT2Tokenizer, GPT2Model
tokenizer = AutoTokenizer.from_pretrained('gpt2')
#model = GPT2Model.from_pretrained(saved_model_path)
text = "this movie is such "
input_ids = tokenizer(text, return_tensors="pt", truncation=True).input_ids
outputs = rlhf_model.generate(input_ids, max_new_tokens=200, do_sample=True,top_p=0.7,temperature=0.1)
output_text = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]
print(output_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


this movie is such  a huge hit that I actually didn't even bother to watch it. 
I actually didn't even bother to watch it. 
I actually didn't even bother to watch it. 
I actually didn't even bother to watch it. 
I actually didn't even bother to watch it. 
I actually didn't even bother to watch it. 
I actually didn't even bother to watch it. 
I actually didn't even bother to watch it. 
I actually didn't even bother to watch it. 
I actually didn't evenprotein.com 
I actually didn't even bother to watch it.
I actually didn't even bother to watch it.
I actually didn't even bother to watch it.
I actually didn't even bother to watch it.
I actually didn't even bother to watch it.
I actually didn't even bother to watch it.
I actually didn't even bother to watch it.



In [ ]:
from transformers import GPT2Tokenizer, GPT2Model,GPT2LMHeadModel
tokenizer = AutoTokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')
text = "this movie is such"
input_ids = tokenizer(text, return_tensors="pt", truncation=True).input_ids
outputs = model.generate(input_ids, max_new_tokens=200, do_sample=True,top_p=0.7,temperature=0.1)
output_text = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]
print(output_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


this movie is such a great movie, I'm going to watch it again. I'm going to watch it again. I'm going to watch it again. I'm going to watch it again. I'm going to watch it again. I'm going to watch it again. I'm going to watch it again. I'm going to watch it again. I'm going to watch it again. I'm going to watch it again. I'm going to watch it again. I'm going to watch it again. I'm going to watch it again. I'm going to watch it again. I'm going to watch it again. I'm going to watch it again. I'm going to watch it again. I'm going to watch it again. I'm going to watch it again. I'm going to watch it again. I'm going towing it again. I'm going towing it again. I'm going towing it again. I'm going towing it again. I'm going to


In [ ]:
print(output_text)

The movie is so much more than a movie. It's a movie about the people who live in the world. It's about the people who live in the world. It's about the people who live in the world. It's about the people who live in the world. It's about the people who live in the world. It's about the people who live in the world. It's about the people who live in the world. It's about the people who live in the world. It's about the people who live in the world. It's about the people who live in the world. It's about the people who live in the world. It's about the people who live in the world. It's about the people who live in the world. It's about the people who live in the world. It's about the people who live in the world. It's about the people who live in the world. It's about the people who live in the world. It's about the people


In [ ]:
%pip install -Uqq git+https://github.com/neelnanda-io/TransformerLens

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/IPython/core/interactiveshell.py:2418 in run_line_magic   │
│                                                                                                  │
│   2415 │   │   │   if getattr(fn, "needs_local_scope", False):                                   │
│   2416 │   │   │   │   kwargs['local_ns'] = self.get_local_scope(stack_depth)                    │
│   2417 │   │   │   with self.builtin_trap:                                                       │
│ ❱ 2418 │   │   │   │   result = fn(*args, **kwargs)                                              │
│   2419 │   │   │   return result                                                                 │
│   2420 │                                                                                         │
│   2421 │   def get_local_scope(self, stack_depth):                                               │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/google/colab/_installation_commands.py:49 in _pip_magic   │
│                                                                                                  │
│   46   # Use bare "pip install" rather than "python -m pip install".                             │
│   47   # Colab is set up such that pip does the right thing, and pip install                     │
│   48   # will properly trigger the pip install warning.                                          │
│ ❱ 49   return ip.system('pip {}'.format(line))                                                   │
│   50                                                                                             │
│   51                                                                                             │
│   52 def _register_magics(ip):                                                                   │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/google/colab/_shell.py:99 in system                       │
│                                                                                                  │
│    96 │   if pip_warn:                                                                           │
│    97 │     kwargs.update({'also_return_output': True})                                          │
│    98 │                                                                                          │
│ ❱  99 │   output = _system_commands._system_compat(self, *args, **kwargs)  # pylint:disable=pr   │
│   100 │                                                                                          │
│   101 │   if pip_warn:                                                                           │
│   102 │     _pip.print_previous_import_warning(output)                                           │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/google/colab/_system_commands.py:453 in _system_compat    │
│                                                                                                  │
│   450   # We set a higher depth than the IPython system command since a shell object             │
│   451   # is expected to call this function, thus adding one level of nesting to the             │
│   452   # stack.                                                                                 │
│ ❱ 453   result = _run_command(                                                                   │
│   454 │     shell.var_expand(cmd, depth=2), clear_streamed_

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
!pip install evaluate

In [ ]:
from typing import List

from datasets import load_dataset
from tqdm import tqdm
from transformers import AutoTokenizer

import trlx
from trlx.data.configs import (
    ModelConfig,
    OptimizerConfig,
    SchedulerConfig,
    TokenizerConfig,
    TrainConfig,
    TRLConfig,
)
from trlx.models.modeling_ppo import PPOConfig

try:
    import evaluate
except ImportError:
    raise ImportError(
        "To run this example, please install the `evaluate` and `nltk` packages" "by running `pip install evaluate`"
    )

config = TRLConfig(
    train=TrainConfig(
        seq_length=612,
        epochs=100,
        total_steps=1000,
        batch_size=1,
        checkpoint_interval=1000,
        eval_interval=10,
        pipeline="PromptPipeline",
        trainer="AcceleratePPOTrainer",
        tracker=None, #default is wandb
    ),
    model=ModelConfig(
        model_path="google/flan-t5-small",
        model_arch_type="seq2seq",
        num_layers_unfrozen=2,
    ),
    tokenizer=TokenizerConfig(
        tokenizer_path="google/flan-t5-small",
        truncation_side="right",
    ),
    optimizer=OptimizerConfig(
        name="adamw",
        kwargs={
            "lr": 1.0e-5,
            "betas": [0.9, 0.999],
            "eps": 1.0e-8,
            "weight_decay": 1.0e-6,
        },
    ),
    scheduler=SchedulerConfig(
        name="cosine_annealing",
        kwargs={
            "T_max": 10000,
            "eta_min": 1.0e-6,
        },
    ),
    method=PPOConfig(
        name="PPOConfig",
        num_rollouts=512,
        chunk_size=12,
        ppo_epochs=4,
        init_kl_coef=0.05,
        target=6,
        horizon=10000,
        gamma=0.99,
        lam=0.95,
        cliprange=0.2,
        cliprange_value=0.2,
        vf_coef=1.0,
        scale_reward=None,
        ref_mean=None,
        ref_std=None,
        cliprange_reward=10,
        gen_kwargs={
            "max_new_tokens": 100,
        },
        gen_experience_kwargs={
            "max_new_tokens": 100,
            "do_sample": True,
            "temperature": 1.0,
            "top_k": 50,
            "top_p": 0.95,
        },
    ),
)


meteor = evaluate.load("meteor")  # use meteor as the reward function



╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 18>:19                                                                            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ModuleNotFoundError: No module named 'evaluate'

During handling of the above exception, another exception occurred:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /usr/local/lib/python3.9/dist-packages/IPython/core/interactiveshell.py:3553 in run_code         │
│                                                                                                  │
│   3550 │   │   │   │   elif async_ :                                                             │
│   3551 │   │   │   │   │   await eval(code_obj, self.user_global_ns, self.user_ns)               │
│   3552 │   │   │   │   else:                                                                     │
│ ❱ 3553 │   │   │   │   │   exec(code_obj, self.user_global_ns, self.user_ns)                     │
│   3554 │   │   │   finally:                                                                      │
│   3555 │   │   │   │   # Reset our crash handler in place                                        │
│   3556 │   │   │   │   sys.excepthook = old_excepthook                                           │
│ in <cell line: 18>:21                                                                            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ImportError: To run this example, please install the `evaluate` and `nltk` packagesby running `pip install 
evaluate`

In [ ]:
start =1

In [ ]:
if start ==1:

    def reward_fn(samples: List[str], prompts: List[str], outputs: List[str]):
        original_summaries = [prompt_label[prompt.strip()] for prompt in prompts]
        scores = [
            meteor.compute(predictions=[output.strip()], references=[original])["meteor"]
            for (original, output) in zip(original_summaries, outputs)
        ]
        return scores

    dataset = load_dataset("cnn_dailymail", "3.0.0", cache_dir="data")

    # take 20,000 samples from the training set as prompts for training
    prompts = dataset["train"]["article"][0:200]
    summaries = dataset["train"]["highlights"][0:200]
    prompts = ["Summarize: " + prompt for prompt in prompts]

    # take 1,000 samples from the validation set as prompts for evaluation
    val_prompts = ["Summarize: " + prompt for prompt in dataset["validation"]["article"][0:10]]
    val_summaries = dataset["validation"]["highlights"][0:10]

    # make dictionary of prompts and labels to use for reward function
    tokenizer = AutoTokenizer.from_pretrained(config.model.model_path)
    tokenizer.padding_side = "left"
    tokenizer.truncation_side = "right"
    tokenizer.sep_token = "<sep>"
    prompt_label = {}
    max_length = config.train.seq_length - config.method.gen_kwargs["max_new_tokens"]

    for i in tqdm(range(len(prompts))):
        key = tokenizer.decode(
            tokenizer(prompts[i], truncation=True, max_length=max_length, add_special_tokens=False)["input_ids"],
            skip_special_tokens=True,
        )  # get prompt like trlx's prompt
        prompt_label[key.strip()] = summaries[i]

    for i in tqdm(range(len(val_prompts))):
        key = tokenizer.decode(
            tokenizer(val_prompts[i], truncation=True, max_length=max_length, add_special_tokens=False)["input_ids"],
            skip_special_tokens=True,
        )  # get prompt like trlx's prompt
        prompt_label[key.strip()] = val_summaries[i]

    trlx.train(
        reward_fn=reward_fn,
        prompts=prompts,
        eval_prompts=val_prompts,
        config=config,
    )

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'start' is not defined